In [2]:
import numpy as np
import numpy.linalg as lg 
from  scipy.io import mmread
from scipy.sparse.linalg import cg
from tabulate import tabulate
A1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())
# A1.shape=(1806, 1806) real symmetric positive definite
A2=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
# A2.shape=(817,817) real symmetric positive definite
A3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
A4=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# A4.shape=(3562,3562) real symmetric positive definite

In [5]:
def cgp_iter(A, b,x0,maxiter,tol,M):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return cg(A, b, callback=callback,tol = tol,maxiter=maxiter,M=lg.inv(M)),num_iters


def Preconditionned_Conjugate_Gradient_Algorithm(A,b,x,itermax,tol):
    M=np.diag(np.diag(A))
    r = b-np.dot(A, x)
    z = p = np.dot(lg.inv(M),r)
    k = 0 
    
    while lg.norm(z) > tol :
        alpha1 = np.dot(r, z)/(np.dot(np.dot(A,p), p))
        x = x + alpha1*p
        rpr=r.copy()
        
        r = rpr - alpha1*np.dot(A, p)
        zpr = z.copy()
        z = np.dot(lg.inv(M),r)
        beta1 = np.dot(r, z)/np.dot(rpr,zpr)
        p = z + beta1*p
        k = k + 1
    return x,k


In [10]:
matricies=[A1,A2,A3,A4]
tol=1e-6
itermax = 400

In [11]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  M=np.diag(np.diag(matricies[i]))
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = cgp_iter(matricies[i],b,x,itermax,tol,M)[1]  
  y=cg(matricies[i],b,x,tol,itermax,M=lg.inv(M))[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  x,iterm  = Preconditionned_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(tol)
  l.append(iterm)
  l.append(res)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','tol','Our_nbr_iter','res','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 

+--------------+-------+----------------+--------------+-------------+------------------+-------------+-------------+
| matricies    |   tol |   Our_nbr_iter |          res |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+==============+=======+================+==============+=============+==================+=============+=============+
| (1806, 1806) | 1e-06 |             87 | 25.9087      | 3.41579e-05 |              400 | 0.000199097 |  8.4885e-11 |
+--------------+-------+----------------+--------------+-------------+------------------+-------------+-------------+
| (817, 817)   | 1e-06 |              1 |  1.20089e-15 | 0           |                1 | 1.20089e-15 |  0          |
+--------------+-------+----------------+--------------+-------------+------------------+-------------+-------------+
| (485, 485)   | 1e-06 |              1 |  8.15844e-16 | 0           |                1 | 8.15844e-16 |  0          |
+--------------+-------+----------------+--------------+